# Using Parallelism with Machine Learning: The Housing Prices Competition 

## Description of the competition

- The Housing Prices Competition train_dataset consists of various features of residential homes in Ames, Iowa, including both quantitative and categorical variables like the size of the property, the number of rooms, year built, and neighborhood quality.
- It includes a set of 79 explanatory variables describing almost every aspect of the houses, allowing for in-depth analysis.
- *The primary goal* of the competition is to predict **the final price of each home**, in this lab we will use *RandomForests*.
- The models are evaluated on Root Mean Squared Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price, encouraging precise predictions over a range of housing prices.

### File descriptions
- *train.csv*: the training set used to train the model.
- *test.csv*: the test set used to compute the performance of the model.
- *train_data_description.txt*: full description of each column.
### Useful train_data fields

Here's a brief version of what you'll find in the train_data description file.

- *SalePrice*: the property's sale price in dollars. This is the target variable that you're trying to predict.
- *MSSubClass*: The building class
- *MSZoning*: The general zoning classification

Teh train_dataset is acessible here: https://www.kaggle.com/code/dansbecker/random-forests/tutorial

## Read and prepare the train_data
*If you're curious about this the professor can explain it for you*.

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the train_dataset
file_path = 'train.csv'
train_data = pd.read_csv(file_path, index_col="Id")

# Columns to be deleted
columns_to_delete = ['MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

# Delete the specified columns
train_data_cleaned = train_data.drop(columns=columns_to_delete, axis=1)

# Define the input features (X) and the output (y)
X = train_data_cleaned.drop('SalePrice', axis=1)
y = train_data_cleaned['SalePrice']

# Identify the categorical columns in X
categorical_columns = X.select_dtypes(include=['object']).columns

# Initialize a LabelEncoder for each categorical column
label_encoders = {column: LabelEncoder() for column in categorical_columns}

# Apply Label Encoding to each categorical column
for column in categorical_columns:
    X[column] = label_encoders[column].fit_transform(X[column])

# Display the first few rows of X to confirm the encoding
print(X.head())


    MSSubClass  MSZoning  LotFrontage  LotArea  Street  LotShape  LandContour  \
Id                                                                              
1           60         3         65.0     8450       1         3            3   
2           20         3         80.0     9600       1         3            3   
3           60         3         68.0    11250       1         0            3   
4           70         3         60.0     9550       1         0            3   
5           60         3         84.0    14260       1         0            3   

    Utilities  LotConfig  LandSlope  ...  GarageQual  GarageCond  PavedDrive  \
Id                                   ...                                       
1           0          4          0  ...           4           4           2   
2           0          2          0  ...           4           4           2   
3           0          4          0  ...           4           4           2   
4           0          0        

## Split the Data into training and test

In [3]:
from sklearn.model_selection import train_test_split

# Split the first dataset (X, y) into train and test sets with a 70% - 30% split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

# Fill NaN values in X_train and X_val with the median of the respective columns
X_train_filled = X_train.fillna(X_train.median())
X_val_filled = X_val.fillna(X_val.median())

(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

((1022, 70), (438, 70), (1022,), (438,))

## First RandomForest Model
This is the code for a simple trial.

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Create a Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the training data
rf_model.fit(X_train_filled, y_train)

# Make predictions on the validation data
y_val_pred_filled = rf_model.predict(X_val_filled)

# Calculate the RMSE on the validation data
rmse_filled = sqrt(mean_squared_error(y_val, y_val_pred_filled))

# Print the RMSE
print(f'RMSE on the validation data: {rmse_filled}')

RMSE on the validation data: 26467.502883356483


### Parameters of Random Forest Model
The three most important parameters that typically have the most impact on the performance of a Random Forest model are:

- *n_estimators*: This parameter specifies the number of trees in the forest. Generally, a higher number of trees increases the performance and makes the predictions more stable, but it also makes the computation slower. Selecting the right number of trees requires balancing between performance and computational efficiency.

- *max_features*: This parameter defines the maximum number of features that are allowed to try in an individual tree. There are several options available for this parameter:

    - *sqrt*: This is commonly used and means that the maximum number of features used at each split is the square root of the total number of features.
    - *log2*: This is another typical option, meaning the log base 2 of the feature count is used.
    - *A specific integer or float*: You can specify an exact number or a proportion of the total.

- *max_depth*: This parameter specifies the maximum depth of each tree. Deeper trees can model more complex patterns, but they also risk overfitting. Limiting the depth of trees can improve the model's generalization and reduce overfitting. It's often useful to set this parameter to a finite value, especially when dealing with a large number of features.

## Finding the best parameters sequentially

In [5]:
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
import time


# Define the parameter ranges
n_estimators_range = [10, 25, 50, 100, 200, 300, 400]
max_features_range = ['sqrt', 'log2', None]  # None means using all features
max_depth_range = [1, 2, 5, 10, 20, None]  # None means no limit


# Function to evaluate all parameter combinations
def evaluate_combinations(n_estimators_range, max_features_range, max_depth_range, X_train_filled, y_train, X_val_filled, y_val):
    best_rmse = float('inf')
    best_mape = float('inf')
    best_model = None
    best_parameters = {}

    for n_estimators in n_estimators_range:
        for max_features in max_features_range:
            for max_depth in max_depth_range:
                # Create and train the Random Forest model
                rf_model = RandomForestRegressor(
                    n_estimators=n_estimators,
                    max_features=max_features,
                    max_depth=max_depth,
                    random_state=42
                )
                rf_model.fit(X_train_filled, y_train)

                # Make predictions and compute RMSE
                y_val_pred = rf_model.predict(X_val_filled)
                rmse = sqrt(mean_squared_error(y_val, y_val_pred))
                mape = mean_absolute_percentage_error(y_val, y_val_pred) * 100

                # Print parameters and metrics
                print(f"The parameters: {n_estimators}, {max_features}, {max_depth}. RMSE: {rmse}, MAPE: {mape}%")

                # Update the best model if this one is better
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_mape = mape
                    best_model = rf_model
                    best_parameters = {
                        'n_estimators': n_estimators,
                        'max_features': max_features,
                        'max_depth': max_depth
                    }

    return best_rmse, best_mape, best_model, best_parameters


# Sequential Execution
start_time = time.time()

best_rmse_seq, best_mape_seq, best_model_seq, best_params_seq = evaluate_combinations(
    n_estimators_range,
    max_features_range,
    max_depth_range,
    X_train_filled,
    y_train,
    X_val_filled,
    y_val
)

end_time = time.time()
sequential_time = end_time - start_time

print(f"\nSequential Execution:")
print(f"Best Parameters: {best_params_seq}, RMSE: {best_rmse_seq}, MAPE: {best_mape_seq}%")
print(f"Execution Time: {sequential_time} seconds")


The parameters: 10, sqrt, 1. RMSE: 62936.129262244904, MAPE: 26.860734529882123%
The parameters: 10, sqrt, 2. RMSE: 51806.105176310826, MAPE: 20.52147680573829%
The parameters: 10, sqrt, 5. RMSE: 34155.52179235385, MAPE: 13.875180501812617%
The parameters: 10, sqrt, 10. RMSE: 30338.849097862072, MAPE: 11.190218719469183%
The parameters: 10, sqrt, 20. RMSE: 29045.13441969273, MAPE: 10.999582155703044%
The parameters: 10, sqrt, None. RMSE: 29857.88816585078, MAPE: 11.14923996984299%
The parameters: 10, log2, 1. RMSE: 64442.107292878405, MAPE: 27.58882909655716%
The parameters: 10, log2, 2. RMSE: 51396.530250771764, MAPE: 21.498982900746558%
The parameters: 10, log2, 5. RMSE: 34660.965531334085, MAPE: 13.934933036220679%
The parameters: 10, log2, 10. RMSE: 29780.885879712325, MAPE: 11.321326655556458%
The parameters: 10, log2, 20. RMSE: 30274.052578164188, MAPE: 11.09443886088674%
The parameters: 10, log2, None. RMSE: 28781.501496629386, MAPE: 11.219414776631876%
The parameters: 10, None,

In [8]:
# Implementing Threads

import threading

# Thread-safe list to store results and lock for printing
lock = threading.Lock()
results_threading = []

# Thread function to evaluate combinations
def thread_worker(param_ranges):
    rmse, mape, model, params = evaluate_combinations(
        param_ranges[0], param_ranges[1], param_ranges[2],
        X_train_filled, y_train, X_val_filled, y_val
    )
    with lock:
        results_threading.append((rmse, mape, params))

# Split parameter ranges into chunks (e.g., by splitting `n_estimators_range`)
chunks = [n_estimators_range[:len(n_estimators_range)//2], n_estimators_range[len(n_estimators_range)//2:]]

threads = []
start_time = time.time()

for chunk in chunks:
    thread = threading.Thread(target=thread_worker, args=([chunk, max_features_range, max_depth_range],))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

end_time = time.time()
threading_time = end_time - start_time

# Find the best result from all threads
best_threading_result = min(results_threading, key=lambda x: x[0])

print(f"\nThreading Execution:")
print(f"Best Parameters: {best_threading_result[2]}, RMSE: {best_threading_result[0]:.4f}, MAPE: {best_threading_result[1]:.2f}%")
print(f"Execution Time: {threading_time:.2f} seconds")


The parameters: 10, sqrt, 1. RMSE: 62936.129262244904, MAPE: 26.860734529882123%
The parameters: 10, sqrt, 2. RMSE: 51806.105176310826, MAPE: 20.52147680573829%
The parameters: 10, sqrt, 5. RMSE: 34155.52179235385, MAPE: 13.875180501812617%
The parameters: 10, sqrt, 10. RMSE: 30338.849097862072, MAPE: 11.190218719469183%
The parameters: 100, sqrt, 1. RMSE: 60537.474850632694, MAPE: 25.057060729227725%
The parameters: 10, sqrt, 20. RMSE: 29045.13441969273, MAPE: 10.999582155703044%
The parameters: 10, sqrt, None. RMSE: 29857.88816585078, MAPE: 11.14923996984299%
The parameters: 10, log2, 1. RMSE: 64442.107292878405, MAPE: 27.58882909655716%
The parameters: 100, sqrt, 2. RMSE: 48526.19663112171, MAPE: 19.149613320776183%
The parameters: 10, log2, 2. RMSE: 51396.530250771764, MAPE: 21.498982900746558%
The parameters: 10, log2, 5. RMSE: 34660.965531334085, MAPE: 13.934933036220679%
The parameters: 10, log2, 10. RMSE: 29780.885879712325, MAPE: 11.321326655556458%
The parameters: 10, log2, 2

In [1]:
# Implementing Processes

from multiprocessing import Pool

# Multiprocessing worker function (evaluates a subset of parameter combinations)
def process_worker(param_ranges):
    return evaluate_combinations(
        param_ranges[0], param_ranges[1], param_ranges[2],
        X_train_filled, y_train, X_val_filled, y_val
    )

# Split parameter ranges into chunks (e.g., by splitting `n_estimators_range`)
chunks = [n_estimators_range[:len(n_estimators_range)//2], n_estimators_range[len(n_estimators_range)//2:]]

start_time = time.time()

with Pool() as pool:
    multiprocessing_results = pool.map(
        process_worker,
        [(chunk, max_features_range, max_depth_range) for chunk in chunks]
    )

end_time = time.time()
multiprocessing_time = end_time - start_time

# Find the best result from all processes
best_multiprocessing_result = min(multiprocessing_results, key=lambda x: x[0])

print(f"\nMultiprocessing Execution:")
print(f"Best Parameters: {best_multiprocessing_result[3]}, RMSE: {best_multiprocessing_result[0]:.4f}, MAPE: {best_multiprocessing_result[1]:.2f}%")
print(f"Execution Time: {multiprocessing_time:.2f} seconds")


NameError: name 'n_estimators_range' is not defined